In [250]:
from datetime impprint_account_historyatetime
from dataclasses import dataclass
from typing import List, Any, Tuple
from  decimal import Decimal
import time
from pydantic import BaseModel


# @dataclass
class MoneyHistory(BaseModel):
    date: datetime
    operation: Decimal



class Account:
    def __init__(self, name, start_balance):
        self.name = name
        self.__balance: Decimal = start_balance
        self.__deposit_history: List[Tuple[MoneyHistory]] = list()
        self.__withdraw_history: List[Tuple[MoneyHistory]] = list()
    
    def __convert_to_money(self,number: Any) -> Decimal:
        try:
            number = Decimal(number)
            if number < 0:
                raise ValueError(f'Number cannot be negative! {number}')
        except:
            print(f'Argument Error {number} is not a number')
        return number
    
    def get_balance(self) -> float:
        """
        return account balace method
        
        params:
        None
        """
        return float(self.__balance) 
    
            
    def withdraw(self, money_to_withdraw: Decimal) -> None:
        """
        withdraw money method
        
        params:
        money_to_withdraw: Decimal
        """
        money_to_withdraw = self.__convert_to_money(money_to_withdraw)
        self.__balance -=  money_to_withdraw
        self.__withdraw_history.append((datetime.now(), -money_to_withdraw))
                                     
    def deposit(self, money_to_deposit: Decimal) -> None:
        """
        deposit money method
        
        params:
        money_to_deposit: Decimal
        """
        money_to_deposit = self.__convert_to_money(money_to_deposit)
        self.__balance += money_to_deposit
        self.__deposit_history.append((datetime.now(), money_to_deposit))
        
    
    def get_account_history(self, descending: bool=True) -> List[MoneyHistory]:
        """
        get_account_history method
        params:
        
        descending: bool  DEFAULT True - return list order by DESC
        """
        acc_history = list()
        acc_history.extend(self.__deposit_history)
        acc_history.extend(self.__withdraw_history)
        acc_history = [(t[0].strftime('%Y-%m-%d %H:%M:%S'), float(t[1])) for t in acc_history]
        return sorted(acc_history, key=lambda x: x[0], reverse=descending)
    
    def print_account_history(self, from_latest: bool=True) -> None:
        """
        print_account_history method
        params:
        
        from_latest: bool  DEFAULT True - print account money change with  order by DESC
        """
        for date, money in daniels_bank_acc.get_account_history(from_latest):
            print(f'At {date}')
            print(f'{self.name}\'s account money were changed by {money}')
            print('')
        print(f'Total {self.name}\'s  account money at {datetime.now().date()}:  {self.get_balance()}')

In [251]:
daniels_bank_acc = Account('DanielAbramov', 100)

daniels_bank_acc.withdraw(50)
daniels_bank_acc.deposit(120)

time.sleep(15)
daniels_bank_acc.deposit(40000)
time.sleep(30)
daniels_bank_acc.withdraw(24000)
time.sleep(10)
daniels_bank_acc.withdraw(2000)

In [252]:
daniels_bank_acc.get_balance()

14170.0

In [253]:
daniels_bank_acc.print_account_history(True)

At 2024-10-05 20:21:28
DanielAbramov's account money were changed by -2000.0

At 2024-10-05 20:21:18
DanielAbramov's account money were changed by -24000.0

At 2024-10-05 20:20:48
DanielAbramov's account money were changed by 40000.0

At 2024-10-05 20:20:33
DanielAbramov's account money were changed by 120.0

At 2024-10-05 20:20:33
DanielAbramov's account money were changed by -50.0

Total DanielAbramov's  account money at 2024-10-05:  14170.0
